In [13]:
# Webスクレイピングに最低限必要なライブラリをインポート
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
from collections import deque # 高速に先頭からデータを取り出す(pop)ために使います
from urllib.parse import urlparse

In [14]:
# アクセスしたいWebサイトのURLを指定
url = 'https://github.com/orgs/google/repositories'

# WebサーバーにHTTPリクエストを送信
# レスポンスを変数に格納しておく
res = requests.get(url)

print(f"ステータスコード:{res.status_code}") # ステータスコードを表示

ステータスコード:200


In [ ]:
# page_data = {}
# visited_url = set()
# visit_url_queue = deque([url])
# base_domain = urlparse(url).netloc


# while visit_url_queue:
#     current_url = visit_url_queue.popleft()
#     if current_url in visited_url:
#         continue

#     visited_url.add(current_url)

#     # ★修正箇所★
#     # どのページを見ているか表示（件数を追加）
#     print(f"[{len(visited_url)}件目] Accessing: {current_url}")

#     try:
#         res = requests.get(current_url)
#         res.encoding = res.apparent_encoding # 文字化け対策
#         time.sleep(1)  # サーバーに負荷をかけないように0.5秒待機

#         content_type = res.headers.get('Content-Type', '')
#         if 'text/html' not in content_type:
#             print(f"  -> スキップ (Content-Type: {content_type})")
#             page_data[current_url] = f"HTMLではない ({content_type})"
#             continue # このページの処理を中断

#         soup = BeautifulSoup(res.text, 'html.parser')

#         title_tag = soup.title
#         if title_tag and title_tag.string:
#             title_text = title_tag.string.strip() # .string で中身を取得し、strip()で前後の空白削除
#         else:
#             title_text = 'タイトルなし' # titleタグがない、または中身が空の場合
        
#         page_data[current_url] = title_text # 辞書に格納

#         links = soup.select('a[href]')

#         found_count = 0
#         for link in links:
#             new_url = urljoin(current_url, link['href'])

#             new_url = new_url.split('#')[0]

#             parsed_new_url = urlparse(new_url)

#             if parsed_new_url.scheme not in ['http', 'https']:
#                 continue



#             if parsed_new_url.netloc == base_domain:
#                 if new_url not in visited_url and new_url not in visit_url_queue:
#                     visit_url_queue.append(new_url)
#                     found_count += 1


#         print(f"Found {found_count} new URLs on {current_url}")

#     except Exception as e:
#         print(f"Error accessing {current_url}: {e}")

# print("\n--- クローリング終了 ---")
# print(f"訪問した総ページ数: {len(visited_url)}")

# # -----------------------------------
# # 課題対応：最終的な辞書を表示
# # -----------------------------------
# print("\n--- 取得した URL と タイトル の辞書 ---")
# # 件数が多いと見にくいため、pprint を使うとより見やすくなります
# # from pprint import pprint
# # pprint(page_data)
# print(page_data)

[1件目] Accessing: https://github.com/orgs/google/repositories
Found 325 new URLs on https://github.com/orgs/google/repositories
[2件目] Accessing: https://github.com/
Found 25 new URLs on https://github.com/
[3件目] Accessing: https://github.com/login?return_to=https%3A%2F%2Fgithub.com%2Forgs%2Fgoogle%2Frepositories
Found 2 new URLs on https://github.com/login?return_to=https%3A%2F%2Fgithub.com%2Forgs%2Fgoogle%2Frepositories
[4件目] Accessing: https://github.com/google
Found 46 new URLs on https://github.com/google
[5件目] Accessing: https://github.com/features/copilot
Found 30 new URLs on https://github.com/features/copilot
[6件目] Accessing: https://github.com/features/spark


KeyboardInterrupt: 

In [ ]:
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
time.sleep(1)




while current_url:
    print(f"Fetching: {current_url}")
    
    # ⭐ ここで遅延を入れるのが正しい配置です ⭐
    time.sleep(2) # 例：2秒待機 (負荷軽減のため必須)
    
    try:
        response = requests.get(current_url)
        response.raise_for_status() # 200 OK以外はエラーにする
        soup = BeautifulSoup(response.content, 'html.parser')

        # 特定のタグをすべて抽出する（例：aタグ）
        names = soup.select('a > span')

        # 結果は Beautiful Soup の Tag オブジェクトのリストになります
        for name_span in names:
            print(name_span.text.strip())
        
        # 処理が進むように今回はNoneで終了させています
        current_url = None 
        
    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        break

print("Finished scraping.")

Fetching: https://github.com/features/spark
GitHub Copilot
GitHub Spark
GitHub Models
Try it now
Try GitHub Spark
Get started
Get started
GitHub on LinkedIn
GitHub on Instagram
GitHub on YouTube
GitHub on X
GitHub on TikTok
GitHub on Twitch
GitHub’s organization on GitHub
English
Português (Brasil)
Español (América Latina)
日本語
한국어
Finished scraping.


In [36]:
import requests
import time
from bs4 import BeautifulSoup
import sqlite3 # データベース処理のために追加

# --- 1. 初期化と初期URL設定 ---
current_url = "/google?tab=repositories" 
all_repositories = [] 

# --- 2. ページネーションを考慮したループ ---
while current_url:
    full_url = "https://github.com" + current_url 
    print(f"Fetching: {full_url}")
    
    # 課題の要件に合わせて time.sleep(1) に修正
    time.sleep(1) 
    
    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # ✅ 修正点1: リポジトリの塊のセレクタをより具体的に指定
        # GitHubの組織リポジトリリストの項目を指すセレクタを使用
        # Googleのページ構造に合わせ、`<div id="org-repositories">` の中の `li` を狙います。
        repo_items = soup.select('#org-repositories li') 
        print(f"Found {len(repo_items)} repository list items (li).") 
        
        # 実際にリポジトリ情報が入っている項目のみを処理
        for item in repo_items:
            # 1. リポジトリ名の抽出
            # ✅ 修正点2: `<h3>` タグの中の `<a>` タグを狙う
            # `h3` の中の `a` タグのテキストがリポジトリ名になっています
            name_tag = item.select_one('h3 a') 
            repo_name = name_tag.text.strip() if name_tag else "N/A"
            
            # 2. 主要言語の抽出
            # ✅ 修正点3: 言語名を示す `span` を狙う
            # 言語名は `span[itemprop="programmingLanguage"]` のテキストとして格納されています
            language_tag = item.select_one('span[itemprop="programmingLanguage"]')
            language = language_tag.text.strip() if language_tag else "N/A"
            
            # 3. スターの数の抽出
            # ✅ 修正点4: スターの数を保持している `a` タグを狙う
            # スター数は、href属性が `/stargazers` で終わる `a` タグのテキストです。
            star_tag = item.select_one('a[href*="/stargazers"]')
            star_count = star_tag.text.strip() if star_tag else "0"
            
            # カンマや k を処理
            if 'k' in star_count:
                 # k の処理（例: '1.2k' -> '1200'）
                if '.' in star_count:
                    # '1.2k' のように小数点がある場合
                    num_str = star_count.replace('k', '').replace(',', '')
                    star_count = str(int(float(num_str) * 1000))
                else:
                    # '1k' のように小数点がない場合
                    star_count = star_count.replace('k', '000').replace(',', '')
            else:
                 # カンマだけを削除
                star_count = star_count.replace(',', '')
            
            # データ型をintに変換できるか確認（文字列として格納するために不要ならスキップ）
            # DBに格納する際にINT型にするために、ここで数値に変換します
            try:
                star_count = int(star_count)
            except ValueError:
                star_count = 0 # 変換できない場合は 0 
            
            all_repositories.append({
                'リポジトリ名': repo_name,
                '主要な言語': language,
                'スターの数': star_count 
            })

        # ページネーションのロジックはそのまま
        next_link = soup.select_one('a.next_page') # 次ページボタンのセレクタもより具体的になりました
        if next_link and next_link.has_attr('href'):
            current_url = next_link['href']
        else:
            current_url = None 

    except requests.exceptions.RequestException as e:
        print(f"Request Error: {e}")
        break
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        break

print("Finished scraping.")
print("-" * 30)

# 最終結果の表示（一部）
print("--- 取得結果（最初の15件）---")
for i, repo in enumerate(all_repositories[:15]): 
    print(repo)
    if i == 14 and len(all_repositories) > 15:
        print('...(以下省略)...')
print("-" * 30)

Fetching: https://github.com/google?tab=repositories
Found 10 repository list items (li).
Finished scraping.
------------------------------
--- 取得結果（最初の15件）---
{'リポジトリ名': 'N/A', '主要な言語': 'C++', 'スターの数': 4984}
{'リポジトリ名': 'N/A', '主要な言語': 'C++', 'スターの数': 608}
{'リポジトリ名': 'N/A', '主要な言語': 'Python', 'スターの数': 166}
{'リポジトリ名': 'N/A', '主要な言語': 'C++', 'スターの数': 3581}
{'リポジトリ名': 'N/A', '主要な言語': 'JavaScript', 'スターの数': 2867}
{'リポジトリ名': 'N/A', '主要な言語': 'C++', 'スターの数': 10266}
{'リポジトリ名': 'N/A', '主要な言語': 'Go', 'スターの数': 158}
{'リポジトリ名': 'N/A', '主要な言語': 'Java', 'スターの数': 1767}
{'リポジトリ名': 'N/A', '主要な言語': 'Java', 'スターの数': 58}
{'リポジトリ名': 'N/A', '主要な言語': 'C++', 'スターの数': 571}
------------------------------
